In [2]:
pip install tensorflow==2.3.2

  Using cached tensorflow-2.3.2-cp37-cp37m-win_amd64.whl (342.5 MB)
  Using cached tensorboard-2.11.2-py3-none-any.whl (6.0 MB)
  Using cached h5py-2.10.0-cp37-cp37m-win_amd64.whl (2.5 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached grpcio-1.62.3-cp37-cp37m-win_amd64.whl (4.5 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached Werkzeug-2.2.3-py3-none-any.whl (233 kB)
  Using cached Markdown-3.4.4-py3-none-any.whl (94 kB)
  Using cached google_auth-2.40.3-py2.py3-none-any.whl (216 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Using cached rsa-4.9.1-py3-none-any.whl (34 kB)
  Using cached cachetools-5.5.2-py3-no

In [44]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical

# Set image dimensions



In [45]:
IMG_SIZE = 224
BATCH_SIZE = 32

# Load and preprocess data
data_dir = "Data"
classes = ['Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag']
X, y = [], []

for idx, label in enumerate(classes):
    folder_path = os.path.join(data_dir, 'train', label)
    for file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        X.append(img)
        y.append(idx)


In [51]:
X=X[:5000]
y=y[:5000]
X = np.array(X, dtype='float32') / 255.0
y = to_categorical(np.array(y))




X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Data augmentation
datagen = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
                             width_shift_range=0.2, height_shift_range=0.2, horizontal_flip=True)


In [52]:

# Model Building
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
outputs = Dense(len(classes), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=outputs)


In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
                    validation_data=(X_val, y_val),
                    epochs=4)
model.save("rice.h5")